In [1]:
from database import Database
from transformators.keyword_transformator import KeywordTransformator
#from transformators.author_transformator import AuthorTransformator
from db_credentials	import dwh_db_connection_params
from database import Database
from variables import sourcepath
import os
import pandas as pd
import numpy as np

In [2]:
from_authors=pd.read_csv(os.path.join(sourcepath, 'authors.csv'))
from_references=pd.read_csv(os.path.join(sourcepath, 'unique_references.csv'))

In [57]:
from_references.head(2)

,citekey,authors,year,title,journal,volume,issue,pages,source_type,publisher,place,editor,monograph_title,note
0,autio_e_information_exposure_opportunity_evalu...,"Autio, E.; Dahlander, L.; Frederiksen, L.",2013,"Information Exposure, Opportunity Evaluation, ...",Acad. Manag. J,56,5,1348-1371,article,NaN,NaN,NaN,NaN,NaN
1,barrett_m_creating_value_in_online_communities...,"Barrett, M.; Oborn, E.; Orlikowski, W.",2016,Creating Value in Online Communities: The Soci...,Inform. Syst. Res,27,4,704-723,article,NaN,NaN,NaN,NaN,NaN


# Transfrom the references to df with the columns "surname" and "lastname"

In [3]:

def split_into_lists_of_two_strings(names):
    """Takes list of names and splits it into sublists of length 2.
    If the length of the split and flattened list is uneven, the last string is dropped.
    
    Args:
        names(list): list of names of multiple authors.

    Returns:
        list of sublists with each sublist containing surname and firstname of one author.
     """
    #split strings upon ' '  and flatten the resulting list of sublists
    names2=[item for sublist in [s.split(' ') for s in names] for item in sublist]
    all=[]
    while len(names2) > 2:
        auth=names2[:2]
        all.append(auth)
        names2=names2[2:]
    return all



#create a new dataframe of authors where each author gets an own row and empty rows are discarded
ref_aut=pd.Series(from_references['authors'].str.split('; ').explode(ignore_index=True).str.split(', ').dropna())

#remove the 'Van' if existing in strings that are longer tan 2, as checks have shown these are most probably parsing errors
ref_aut.apply(lambda l: (l.remove('Van') if 'Van' in l else l) if len(l)>2 else l)

#df_ex_aut.apply(lambda l: (split_into_lists_of_two_strings([item for sublist in [s.split(' ') for s in l] for item in sublist])) if len(l)>2 else l)

#keep only the rows with a length of 2 now:
keep=ref_aut[ref_aut.str.len()==2]

#those that are longer must be changed
change1=ref_aut[ref_aut.str.len()>2]
#first split every string in list from each other, then split into sublist pairs of two
change1=change1.apply(lambda l: split_into_lists_of_two_strings(l))
#and append now corrected series again to keep
keep.append(change1.explode())

#those that are shorter must be changed
change2=ref_aut[ref_aut.str.len()<2]
#those that are shorter than 2 letters can be dropped
change2=change2[change2.explode().str.len()>2]
#many rows contain names that are just not separated by comma and therefore not recognized. lets split them into sublist of two strings each
change2=change2.apply(lambda l: split_into_lists_of_two_strings(l))
#unnest lists to new rows
change2=change2.explode()
#keep only those that contain two strings
change2=change2[change2.str.len()==2]
#and append the transformed rows to keep
keep.append(change2)

#generate a new dataframe
ref_aut_df=pd.DataFrame(keep)
#split up authors column into surname and firstname
ref_aut_df[['surname', 'firstname']]=pd.DataFrame(ref_aut_df.authors.tolist(), index=ref_aut_df.index)
#keep only those two columns
reference_authors=ref_aut_df[['surname', 'firstname']]

#last, remove duplicate authors
reference_authors.drop_duplicates(inplace=True, ignore_index=True)

reference_authors.head(3)


/home/luisa_waack/virtualenvs/thesis3.9/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,surname,firstname
0,Autio,E.
1,Dahlander,L.
2,Frederiksen,L.


In [4]:
from gensim.parsing.preprocessing import strip_numeric
import re

def remove_numbers_amp_and_at_tags(fullname):
    """removes numbers, &amp tags, @institutions, semicolons, dots and round brackets.
    
    Args:
        fullname(str): the fullname of an author entry.
    
    Returns:
        The cleaned string.
    """
    fullname=strip_numeric(fullname)
    fullname=re.sub('&\w+', '', fullname)
    fullname=re.sub('@\w+', '', fullname)
    fullname=re.sub('\| ', '', fullname)
    fullname=re.sub('[;().|]', '', fullname)
    fullname=fullname.strip()
    return fullname

def split_fullname(fullname):
    """Splits a fullname into first-, middle- and surname.
    
    Args:
        fullname(str): the string of the fullname.
    
    Returns:
        The strings surname, firstname and middlename.
    """
    fn_list=fullname.split(', ')
    fn_surname=fn_list[0]

    if len(fn_list)>1:
        first_middle=fn_list[1].split(' ')
        firstname=first_middle[0]
        if len(first_middle)>1:
            middlename=first_middle[1]
        else:
            middlename=np.nan
    else:
        firstname=np.nan
        middlename=np.nan
    return fn_surname, firstname, middlename
    

In [254]:
from_authors=pd.read_csv(os.path.join(sourcepath, 'authors.csv'))

In [5]:

from_authors.fullname=from_authors.fullname.apply(lambda fn: remove_numbers_amp_and_at_tags(fn))
from_authors[['surname', 'firstname', 'middlename']]=pd.DataFrame(from_authors.fullname.apply(lambda fn: split_fullname(fn)).to_list(), index=from_authors.index)


In [256]:
from_authors.head(10)

,article_id,author_position,fullname,surname,firstname,middlename,email,departments,institutions,countries
0,1,1,"Yan, Kevin",Yan,Kevin,NaN,NaN,NaN,Dalton State College,United States
1,1,2,"Leidner, Dorothy E",Leidner,Dorothy,E,NaN,NaN,Baylor University and Lund University,United States
2,1,3,"Zou, Weifei",Zou,Weifei,NaN,NaN,NaN,Deakin University,Australia
3,2,1,"Wilson, F A",Wilson,F,A,NaN,The Information Technology Institute,University of Salford,UK
4,2,2,"Galliers, R D",Galliers,R,D,NaN,The Information Technology Institute,University of Salford,UK
5,3,1,"Aaen, Ivan",Aaen,Ivan,NaN,aaen@acm.org,Department of Computer Science,Aalborg University,Denmark
6,4,1,"Aakhus, Mark",Aakhus,Mark,NaN,aakhus@rutgers.edu,School of Communication and Information,Rutgers University,U.S.A
7,4,2,"Ågerfalk, Pär J",Ågerfalk,Pär,J,par.agerfalk@im.uu.se,Department of Informatics and Media,Uppsala University,SWEDEN
8,4,3,"Lyytinen, Kalle",Lyytinen,Kalle,NaN,NaN,"Weatherhead School of Management, Case Western...",NaN,U.S.A
9,4,4,"Teeni, Dov",Teeni,Dov,NaN,teeni@tau.ac.il,Recanati Business School,Tel Aviv University,ISRAEL


In [257]:
type(from_authors.middlename[0])

float

In [6]:
def try_impute_missing (ent):
    try:
        return ent[0]
    except:
        return np.nan

In [7]:
aggregate_functions={'surname': 'first', 
'firstname': 'first', 
'middlename': 'first', 
'email': lambda x: try_impute_missing(x.mode()),
'departments': lambda x: try_impute_missing(x.mode()), 
'institutions': lambda x: try_impute_missing(x.mode()), 
'countries': lambda x: try_impute_missing(x.mode())}
from_authors=from_authors.groupby('fullname')[['firstname', 'middlename', 'surname', 'email','departments', 'institutions', 'countries']].agg(aggregate_functions)

In [8]:
test_aut=from_authors.iloc[710:720]
test_aut

,surname,firstname,middlename,email,departments,institutions,countries
fullname,,,,,,,
"Bengtsson, Fredrik",Bengtsson,Fredrik,None,NaN,Department of Informatics and Media,Uppsala University,Sweden
"Benitez, Jose",Benitez,Jose,None,jose.benitez@rennes-sb.com,Department of Management,University of Granada,Spain
"Benitez-Amado, Jose",Benitez-Amado,Jose,None,joseba@ugr.es,Department of Management Science,University of Granada,Spain
"Benjamin Lowry, Paul",Benjamin Lowry,Paul,None,NaN,NaN,NaN,NaN
"Benjamin, Paul",Benjamin,Paul,None,NaN,NaN,NaN,NaN
"Benjamin, Victor",Benjamin,Victor,None,NaN,NaN,NaN,NaN
"Benlian,","Benlian,",None,None,NaN,NaN,NaN,NaN
"Benlian, Alexander",Benlian,Alexander,None,benlian@ise.tu-darmstadt.de,Chair of Information Systems and Electronic Se...,Technische Universität Darmstadt,Germany
"Benner, Mary J",Benner,Mary,J,NaN,Carlson School of Management,University of Minnesota,U.S.A


In [9]:
test_ref=reference_authors.head(10)
test_ref

,surname,firstname
0,Autio,E.
1,Dahlander,L.
2,Frederiksen,L.
3,Barrett,M.
4,Oborn,E.
5,Orlikowski,W.
6,Benlian,A.
7,Hess,T.
8,Blanchard,A.
9,Borgatti,S.


In [49]:
#simulate finding delta between source and db data
source_aut=test_aut.append(test_ref, sort=False, ignore_index=True)

db_aut=from_authors[715:731]


In [50]:
source_aut['row_effective_date']=pd.to_datetime('today').normalize()
source_aut['row_expiration_date']=pd.Timestamp.max.normalize()
source_aut['current_row_indicator']='Current'
#other would be 'Expired'
source_aut.head()

,surname,firstname,middlename,email,departments,institutions,countries,row_effective_date,row_expiration_date,current_row_indicator
0,Bengtsson,Fredrik,None,NaN,Department of Informatics and Media,Uppsala University,Sweden,2021-12-07,2262-04-11,Current
1,Benitez,Jose,None,jose.benitez@rennes-sb.com,Department of Management,University of Granada,Spain,2021-12-07,2262-04-11,Current
2,Benitez-Amado,Jose,None,joseba@ugr.es,Department of Management Science,University of Granada,Spain,2021-12-07,2262-04-11,Current
3,Benjamin Lowry,Paul,None,NaN,NaN,NaN,NaN,2021-12-07,2262-04-11,Current
4,Benjamin,Paul,None,NaN,NaN,NaN,NaN,2021-12-07,2262-04-11,Current


In [51]:
db_aut['row_effective_date']=pd.to_datetime('2021-11-27', infer_datetime_format=True).normalize()
db_aut['row_expiration_date']=pd.Timestamp.max.normalize()
db_aut['current_row_indicator']='Current'
db_aut.head()

/tmp/ipykernel_3102478/1083042500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_aut['row_effective_date']=pd.to_datetime('2021-11-27', infer_datetime_format=True).normalize()
/tmp/ipykernel_3102478/1083042500.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_aut['row_expiration_date']=pd.Timestamp.max.normalize()
/tmp/ipykernel_3102478/1083042500.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,surname,firstname,middlename,email,departments,institutions,countries,row_effective_date,row_expiration_date,current_row_indicator
fullname,,,,,,,,,,
"Benjamin, Victor",Benjamin,Victor,None,NaN,NaN,NaN,NaN,2021-11-27,2262-04-11,Current
"Benlian,","Benlian,",None,None,NaN,NaN,NaN,NaN,2021-11-27,2262-04-11,Current
"Benlian, Alexander",Benlian,Alexander,None,benlian@ise.tu-darmstadt.de,Chair of Information Systems and Electronic Se...,Technische Universität Darmstadt,Germany,2021-11-27,2262-04-11,Current
"Benner, Mary J",Benner,Mary,J,NaN,Carlson School of Management,University of Minnesota,U.S.A,2021-11-27,2262-04-11,Current
"Bennett, J",Bennett,J,None,rbennett@latech.edu,"Department of Management, College of Administr...",Louisiana Tech University,"USA,",2021-11-27,2262-04-11,Current


In [14]:
#which authors appear in the source data and as well in the DWH data
intersection=pd.merge(source_aut, db_aut, how = 'inner', on=['surname', 'firstname', 'middlename'])
intersection

,surname,firstname,middlename,email_x,departments_x,institutions_x,countries_x,row_effective_date_x,row_expiration_date_x,current_row_indicator_x,email_y,departments_y,institutions_y,countries_y,row_effective_date_y,row_expiration_date_y,current_row_indicator_y
0,Benjamin,Victor,None,NaN,NaN,NaN,NaN,2021-12-07,2262-04-11,Current,NaN,NaN,NaN,NaN,2021-11-27,2262-04-11,Current
1,"Benlian,",None,None,NaN,NaN,NaN,NaN,2021-12-07,2262-04-11,Current,NaN,NaN,NaN,NaN,2021-11-27,2262-04-11,Current
2,Benlian,Alexander,None,benlian@ise.tu-darmstadt.de,Chair of Information Systems and Electronic Se...,Technische Universität Darmstadt,Germany,2021-12-07,2262-04-11,Current,benlian@ise.tu-darmstadt.de,Chair of Information Systems and Electronic Se...,Technische Universität Darmstadt,Germany,2021-11-27,2262-04-11,Current
3,Benner,Mary,J,NaN,Carlson School of Management,University of Minnesota,U.S.A,2021-12-07,2262-04-11,Current,NaN,Carlson School of Management,University of Minnesota,U.S.A,2021-11-27,2262-04-11,Current
4,Bennett,J,None,rbennett@latech.edu,"Department of Management, College of Administr...",Louisiana Tech University,"USA,",2021-12-07,2262-04-11,Current,rbennett@latech.edu,"Department of Management, College of Administr...",Louisiana Tech University,"USA,",2021-11-27,2262-04-11,Current


In [41]:
intersection.iloc[2].to_dict()

{'surname': 'Benlian',
 'firstname': 'Alexander',
 'middlename': None,
 'email_x': 'benlian@ise.tu-darmstadt.de',
 'departments_x': 'Chair of Information Systems and Electronic Services',
 'institutions_x': 'Technische Universität Darmstadt',
 'countries_x': 'Germany',
 'row_effective_date_x': Timestamp('2021-12-07 00:00:00'),
 'row_expiration_date_x': Timestamp('2262-04-11 00:00:00'),
 'current_row_indicator_x': 'Current',
 'email_y': 'benlian@ise.tu-darmstadt.de',
 'departments_y': 'Chair of Information Systems and Electronic Services',
 'institutions_y': 'Technische Universität Darmstadt',
 'countries_y': 'Germany',
 'row_effective_date_y': Timestamp('2021-11-27 00:00:00'),
 'row_expiration_date_y': Timestamp('2262-04-11 00:00:00'),
 'current_row_indicator_y': 'Current'}

In [52]:
#for testing SCD 2
row_change= {'surname': 'Benlian',
    'firstname': 'Alexander',
    'middlename': None,
    'email': 'benlian@ise.tu-münchen.de',
    'departments': 'Chair of Information Systems and Electronic Services',
    'institutions': 'Technische Universität München',
    'countries_x': 'Germany',
    'row_effective_date': '2021-12-07 00:00:00',
    'row_expiration_date': '2262-04-11 00:00:00',
    'current_row_indicator': 'Current'
}

source_aut=source_aut.append(row_change, ignore_index=True)


,surname,firstname,middlename,email,departments,institutions,countries,row_effective_date,row_expiration_date,current_row_indicator,countries_x
0,Bengtsson,Fredrik,None,NaN,Department of Informatics and Media,Uppsala University,Sweden,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN
1,Benitez,Jose,None,jose.benitez@rennes-sb.com,Department of Management,University of Granada,Spain,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN
2,Benitez-Amado,Jose,None,joseba@ugr.es,Department of Management Science,University of Granada,Spain,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN
3,Benjamin Lowry,Paul,None,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN
4,Benjamin,Paul,None,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN
5,Benjamin,Victor,None,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN
6,"Benlian,",None,None,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN
7,Benlian,Alexander,None,benlian@ise.tu-darmstadt.de,Chair of Information Systems and Electronic Se...,Technische Universität Darmstadt,Germany,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN
8,Benner,Mary,J,NaN,Carlson School of Management,University of Minnesota,U.S.A,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN
9,Bennett,J,None,rbennett@latech.edu,"Department of Management, College of Administr...",Louisiana Tech University,"USA,",2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN


In [66]:
#test SCD 1 (overwrite changed emails)
# source_aut.iloc[9].to_dict()
scd1_row={'surname': 'Bennett',
 'firstname': 'J',
 'middlename': None,
 'email': 'rbennett@gmail.com',
 'departments': 'Department of Management, College of Administration and Business',
 'institutions': 'Louisiana Tech University',
 'countries': 'USA,',
 'row_effective_date': '2021-12-07 00:00:00',
 'row_expiration_date': '2262-04-11 00:00:00',
 'current_row_indicator': 'Current',
 'countries_x': np.nan}

source_aut=source_aut.append(scd1_row, ignore_index=True)


In [67]:
#perform a left join on the fresh source data and the DWH data on the author names
left=pd.merge(source_aut, db_aut, how = 'left', on=['surname', 'firstname', 'middlename'], suffixes=[None, '_db'])
#get the rows that were not previously present in the DWH
ready_to_load=left[left.current_row_indicator_db.isna()][['surname', 'firstname', 'middlename', 'email', 'departments', 'institutions', 'countries', 'row_effective_date', 'row_expiration_date', 'current_row_indicator']]

#this can already be written to the DWH
ready_to_load

,surname,firstname,middlename,email,departments,institutions,countries,row_effective_date,row_expiration_date,current_row_indicator
0,Bengtsson,Fredrik,None,NaN,Department of Informatics and Media,Uppsala University,Sweden,2021-12-07 00:00:00,2262-04-11 00:00:00,Current
1,Benitez,Jose,None,jose.benitez@rennes-sb.com,Department of Management,University of Granada,Spain,2021-12-07 00:00:00,2262-04-11 00:00:00,Current
2,Benitez-Amado,Jose,None,joseba@ugr.es,Department of Management Science,University of Granada,Spain,2021-12-07 00:00:00,2262-04-11 00:00:00,Current
3,Benjamin Lowry,Paul,None,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current
4,Benjamin,Paul,None,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current
10,Autio,E.,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current
11,Dahlander,L.,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current
12,Frederiksen,L.,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current
13,Barrett,M.,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current
14,Oborn,E.,NaN,NaN,NaN,NaN,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current


In [80]:
#get the rows that were in the DWH before already
maybe_changed=left[~left.current_row_indicator_db.isna()]
#check whether the information about 'departments', 'institutions', 'countries' in the intersecting rows has really changed and if it is not just all NaN - here we would want to add a new row (type 2 SCD)
SCD2_change=maybe_changed[((maybe_changed['departments']!=maybe_changed['departments_db']) | (maybe_changed['institutions']!=maybe_changed['institutions_db']) | (maybe_changed['countries']!=maybe_changed['countries_db'])) 
& (~maybe_changed[['departments', 'institutions', 'countries', 'departments_db', 'institutions_db', 'countries_db']].isnull().all(1))]

#here we will change in the existing row the expiration date to today and mark the entry as 'Expired', then add a new row with the fresh institute data as it is in the source
SCD2_change

,surname,firstname,middlename,email,departments,institutions,countries,row_effective_date,row_expiration_date,current_row_indicator,countries_x,email_db,departments_db,institutions_db,countries_db,row_effective_date_db,row_expiration_date_db,current_row_indicator_db
20,Benlian,Alexander,None,benlian@ise.tu-münchen.de,Chair of Information Systems and Electronic Se...,Technische Universität München,NaN,2021-12-07 00:00:00,2262-04-11 00:00:00,Current,Germany,benlian@ise.tu-darmstadt.de,Chair of Information Systems and Electronic Se...,Technische Universität Darmstadt,Germany,2021-11-27,2262-04-11,Current


In [ ]:
for index, row in SCD2_change.iterrows():
    sql='update dim_author set departments = %s, institutions = %s, countries = %s where surname = %s, firstname = %s, middlename = %s'
    cur

In [82]:
SCD2_change.index

Int64Index([20], dtype='int64')

In [72]:
maybe_changed.index

Int64Index([5, 6, 7, 8, 9, 20, 21], dtype='int64')

In [81]:
#find changes in email but no other changes
SCD1_change=maybe_changed[(maybe_changed['email']!=maybe_changed['email_db']) 
& (~maybe_changed.index.isin(SCD2_change.index))
& (~maybe_changed['email'].isnull())]

#here we will just overwrite the email with the email from source data
SCD1_change

,surname,firstname,middlename,email,departments,institutions,countries,row_effective_date,row_expiration_date,current_row_indicator,countries_x,email_db,departments_db,institutions_db,countries_db,row_effective_date_db,row_expiration_date_db,current_row_indicator_db
21,Bennett,J,None,rbennett@gmail.com,"Department of Management, College of Administr...",Louisiana Tech University,"USA,",2021-12-07 00:00:00,2262-04-11 00:00:00,Current,NaN,rbennett@latech.edu,"Department of Management, College of Administr...",Louisiana Tech University,"USA,",2021-11-27,2262-04-11,Current


In [304]:
source_aut[source_aut['surname'].isin(db_aut['surname'])].head()

,surname,firstname,middlename,email,departments,institutions,countries,firstname_initial,row_effective_date,row_expiration_date,current_row_indicator
4,Benjamin,Paul,None,NaN,NaN,NaN,NaN,P.,2021-12-06,2262-04-11,Current
5,Benjamin,Victor,None,NaN,NaN,NaN,NaN,V.,2021-12-06,2262-04-11,Current
6,"Benlian,",None,None,NaN,NaN,NaN,NaN,NaN,2021-12-06,2262-04-11,Current
7,Benlian,Alexander,None,benlian@ise.tu-darmstadt.de,Chair of Information Systems and Electronic Se...,Technische Universität Darmstadt,Germany,A.,2021-12-06,2262-04-11,Current
8,Benner,Mary,J,NaN,Carlson School of Management,University of Minnesota,U.S.A,M.,2021-12-06,2262-04-11,Current


In [298]:
pd

,surname,firstname,middlename,email,departments,institutions,countries,firstname_initial,row_effective_date,row_expiration_date,current_row_indicator
0,Bengtsson,Fredrik,None,NaN,Department of Informatics and Media,Uppsala University,Sweden,F.,2021-12-06,2262-04-11,Current
1,Benitez,Jose,None,jose.benitez@rennes-sb.com,Department of Management,University of Granada,Spain,J.,2021-12-06,2262-04-11,Current
2,Benitez-Amado,Jose,None,joseba@ugr.es,Department of Management Science,University of Granada,Spain,J.,2021-12-06,2262-04-11,Current
3,Benjamin Lowry,Paul,None,NaN,NaN,NaN,NaN,P.,2021-12-06,2262-04-11,Current
4,Benjamin,Paul,None,NaN,NaN,NaN,NaN,P.,2021-12-06,2262-04-11,Current
5,Benjamin,Victor,None,NaN,NaN,NaN,NaN,V.,2021-12-06,2262-04-11,Current
6,"Benlian,",None,None,NaN,NaN,NaN,NaN,NaN,2021-12-06,2262-04-11,Current
7,Benlian,Alexander,None,benlian@ise.tu-darmstadt.de,Chair of Information Systems and Electronic Se...,Technische Universität Darmstadt,Germany,A.,2021-12-06,2262-04-11,Current
8,Benner,Mary,J,NaN,Carlson School of Management,University of Minnesota,U.S.A,M.,2021-12-06,2262-04-11,Current
9,Bennett,J,None,rbennett@latech.edu,"Department of Management, College of Administr...",Louisiana Tech University,"USA,",J.,2021-12-06,2262-04-11,Current
